In [16]:
import pandas as pd

# Load the Excel files
Brasileiros = "Brasileiros.xlsx"
Gringos = "Gringos.xlsx"

df1 = pd.read_excel(Brasileiros)
df2 = pd.read_excel(Gringos)

# Compare the dataframes
#comparison_result = df1.compare(df2)

# Display the comparison result
#print(comparison_result)


In [18]:
activity_mapping = {
    "Música": "Music (Concerts, Music Festivals)",
    "Arte local": "Art (Museums, Galleries, Street Art)",
    "Culinária local": "Food (Local Cuisine, Food Festivals)",
    "Atividades ao Ar Livre": "Outdoor Activities (Hiking, Picnics)",
    "Esportes": "Sports (Soccer, Basketball, Running)"
}

# Mapping of other column options from Portuguese to English
other_mapping = {
    "Sim": "Yes",
    "Não": "No",
    "Talvez": "Maybe",
    "Masculino": "Male",
    "Feminino": "Female"
}

# Mapping of exchange countries from Portuguese to English
country_mapping = {
    "Holanda": "Netherlands",
    "França": "France",
    "Dinamarca": "Denmark",
    "Suécia": "Sweden",
    "Espanha": "Spain",
    "Moçambique": "Mozambique",
    "Portugal": "Portugal",
    "Alemanha": "Germany",
    "Canadá": "Canada",
    "Noruega": "Norway",
    "Grécia": "Greece",
    "Itália": "Italy"
}

language_mapping = {
    "Português": "Portuguese",
    "Inglês": "English",
    "Espanhol": "Spanish",
    "Francês": "French",
    "Italiano": "Italian",
    "Alemão": "German",
    "Holandês": "Dutch",
    "Sueco": "Swedish",
    "Dinamarquês": "Danish",
    "Norueguês": "Norwegian",
    "Grego": "Greek",
    "Mandarim": "Mandarin",
    "Japonês": "Japanese",
    "Coreano": "Korean",
    "Árabe": "Arabic"
}

In [28]:
# Define a function to translate and compare options
def translate_and_compare(option, mapping, options):
    translated_option = mapping.get(option, option)
    return translated_option in options

# Define a function to calculate match score
def calculate_match_score(row1, row2):
    match_score = 0
    
    # Compare languages spoken
    languages1 = set(row1["Quais línguas você fala?"].split("; ")) if isinstance(row1["Quais línguas você fala?"], str) else set()
    
    # Check if the language value is a string before splitting
    languages2 = set(row2["Which languages do you speak?"].split(";")) if isinstance(row2["Which languages do you speak?"], str) else set()
    
    translated_languages1 = {language_mapping.get(lang, lang) for lang in languages1}
    
    if translated_languages1 & languages2:
        match_score += 2
    
    # Compare participation in Economíadas
    if translate_and_compare(row1["Vai no economíadas?"], other_mapping, ["Yes"]):
        match_score += 3
    
    # Compare exchange country preference
    preferred_countries1 = row1["Pretende fazer intercâmbio? Se sim, para qual país?"].split("; ")
    preferred_country2 = country_mapping.get(row2["Country"], "")
    if preferred_country2 in preferred_countries1:
        match_score += 5
    
    # Compare cultural or social activities preference
    activities1 = set(row1.get("Quais das seguintes atividades culturais ou sociais você gostaria de participar? (Selecione todas as que se aplicam)", "").split("; "))
    activities2 = set(row2.get("Which of the following cultural or social activities would you like to engage in during your exchange? (Select all that apply)", "").split(";"))
    translated_activities1 = set(activity_mapping.get(activity, activity) for activity in activities1)
    
    if translated_activities1 & activities2:
        match_score += 4
    
    return match_score

# Create a list to store match results
matches = []

# Iterate through rows in df1 and find the best match from df2
for index1, row1 in df1.iterrows():
    best_match = None
    best_score = 0
    
    for index2, row2 in df2.iterrows():
        score = calculate_match_score(row1, row2)
        
        if score > best_score:
            best_score = score
            best_match = index2
    
    if best_match is not None:
        matches.append((row1["Nome completo"], df2.loc[best_match, "Full name"], best_score))


In [32]:
# Create a dictionary to store match results for people in df1
matches_df1 = {}

# Iterate through rows in df1 and find up to 4 best matches from df2
for index1, row1 in df1.iterrows():
    best_matches = [None] * 4  # Store up to 4 best matches for each person in df1
    best_scores = [0] * 4  # Store scores corresponding to each best match

    for index2, row2 in df2.iterrows():
        score = calculate_match_score(row1, row2)

        # Only consider the score if it's greater than the lowest score in best_scores
        if score > min(best_scores):
            min_score_index = best_scores.index(min(best_scores))
            best_scores[min_score_index] = score
            best_matches[min_score_index] = index2

    # Filter out None values and store the matches
    valid_matches = [(df2.loc[match, "Full name"], score) for match, score in zip(best_matches, best_scores) if match is not None]
    if valid_matches:
        matches_df1[index1] = valid_matches


# Create a dictionary to store the single best match for people in df2
matches_df2 = {}

# Iterate through rows in df2 and find the best match from df1
for index2, row2 in df2.iterrows():
    best_match = None
    best_score = 0

    for index1, row1 in df1.iterrows():
        score = calculate_match_score(row1, row2)

        if score > best_score:
            best_score = score
            best_match = index1

    if best_match is not None:
        matches_df2[index2] = (df1.loc[best_match, "Nome completo"], best_score)

# Display the matches for people in df1
for idx1, matches in matches_df1.items():
    print(f"Matches for {df1.loc[idx1, 'Nome completo']}:")
    for match in matches:
        print(f"- {match[0]} with score {match[1]}")

# Display the matches for people in df2
for idx2, match in matches_df2.items():
    print(f"Match for {df2.loc[idx2, 'Full name']} with {match[0]} with score {match[1]}")

Matches for Gabrielle Orso:
- Marcus Thor Denman with score 9
- Blanca Nevado with score 9
- Manal Chaffoui with score 9
- Emily Sophie wessling with score 9
Matches for Esther Caroline Cunha Rodrigues:
- Gonçalo Abdula Franco  with score 6
- Raphael Emeraud with score 4
- Guénolé Querelle with score 4
- Emily Sophie wessling with score 4
Matches for Maria Fernanda de Almeida Castro Amorim:
- Marcus Thor Denman with score 6
- Manal Chaffoui with score 6
- Ryan den dekker with score 6
- Emily Sophie wessling with score 6
Matches for Davi Cunha Magalhães:
- Blanca Nevado with score 9
- Ana Encinas Sánchez with score 9
- Daniel Piña Ahrends  with score 9
- Emily Sophie wessling with score 7
Matches for Giovanna Giorgi Manente:
- Marcus Thor Denman with score 9
- Manal Chaffoui with score 9
- Ryan den dekker with score 9
- Emily Sophie wessling with score 9
Matches for Pedro Toledo Piza Civita:
- Marcus Thor Denman with score 6
- Blanca Nevado with score 6
- Manal Chaffoui with score 6
- E